# <font color = "red">[READ THIS] Before you start</font>
Welcome to UpLevel mini-projects! In this series, you're challenged to independently work (with our guidance) with data that you will collect to UpLevel yourself. 

We hope you're excited to embark on this adventure.
### Warning:
This isn't just any coding course or programmes, where you receive helper code as you move from one code block to another. 

This is when things get <em>real</em>.

In this project, you will receive instructions to execute a task along with an intended outcome. Most importantly, we will provide you with directions you can go to, to pick up code independently and implement it in this notebook.

Don't worry though, we'll be dropping lots of resources you can consult and these readings will contain everything you need to succeed. You can also perform independent research to find answers independently. You just have to read closely and pick out the parts that make most sense. 

<strong>We make you do this not because we're lazy bastards but because being able to indepedently find code is a highly underrated skill and that's something all companies look out for.</strong>

If you're really stuck and are on the verge of giving up, we gotchu fam. Head on over to our Telegram community at https://bit.ly/UpLevelSG and post your questions there. 

### What we'll be doing:
In this project, we will do the following:

1. <font color='orange'>[Google Colab]</font> Scrape and collect news articles from NewsWeek (Part I)
2. <font color='orange'>[Google Colab]</font> Perform extractive summarization on collected articles (Part II) 
3. <font color='orange'>[Google Colab]</font> Perform abstractive summarization on collected articles (Part III) 
4. <font color='orange'>[Google Colab]</font> Comparing and analyzing the differences between the two kinds of summaries (Part IV)

### Expectations:
We're not going to sugarcoat it - it'll be challenging at times. You have to promise to put in the time and effort to UpLevel yourself. 

But we promise you that it'll ultimately be fun and rewarding, and you'll come out of it stronger and more confident than before.

# Introduction
<font color='green'>Summarizing notes and articles can be a drag, especially when you have to go through the entire document repeatedly to identify important point.</font>

<font color='green'>You remember reading about summarization techniques in Natural Language Processing, which you think can be applied in your day-to-day.</font>

<font color='green'>As such, you decided to try the new techniques on news articles that you intend to collect by yourself on Newsweek. 
</font>

In this Part, we will do the following:
1. Explore the HTML of Newsweek 
2. Scrape NewsWeek articles and summaries

In this Part, we will teach you to scrape a very small number of articles from NewsWeek. We will then provide you with a pre-scraped CSV for subsequent portions. 

The reason for that was that we do not want to overload their servers. The main point of this Part is to teach you the principles of scraping and the thought processes behind it. 

As such, if you're already familiar with scraping and want to skip ahead, just click <a href="https://uplevelsg.s3-ap-southeast-1.amazonaws.com/df_part_i.csv">here</a> to acquire the data.

---

<em>Disclaimer

If the scraping portion doesn't work due to a change in Newsweek's HTML structure, and you got this notebook specifically because of the exciting scraping part - we're really sorry.

Drop Jackie an email at jackie@uplevel.work and we'll do our best to either replace this project with another or issue you a full refund.</em>

### Step 1: Import libraries
We'll be importing a few libraries for scraping:
- requests
- bs4
- pandas as pd

Useful reading for scraping: https://www.dataquest.io/blog/web-scraping-beautifulsoup/

In [1]:
# Step 1: Import your libraries
import requests
import bs4
import pandas as pd

# Structure of NewsWeek
We will be scraping news articles from Newsweek (https://www.newsweek.com/newsfeed). NewsWeek

![NewsWeekStructure](https://uplevelsg.s3-ap-southeast-1.amazonaws.com/ProjectTextSummarizer/NewsWeekStructure.png)
There are three components involved in the scraping:
1. Listing pages containing article summaries
2. Article summaries
3. Article text

In our scraping campaign, we will first go through pages of listings (1) containing summaries (2), and then collect the corresponding articles (3) within each article URL.

# Test scraping one listings page
We'll start with scraping the items from the first page (https://www.newsweek.com/newsfeed?page=1).

Here's an example of 1 "box".

![ListingItem](https://uplevelsg.s3-ap-southeast-1.amazonaws.com/ProjectTextSummarizer/ListingItem.png)

1. category
2. url
3. title
4. summary

Each listings page contains 30 summaries but we'll start slowly first.

### Step 2: Get the response of the first page of Newsweek listing
We'll start with getting the response of the GET request that we make for "https://www.newsweek.com/newsfeed?page=1"

Assign the results to a variable and check for the response code.

Make sure it's 200! 

In [213]:
# Step 2: Make a request and check the response code
url = "https://www.newsweek.com/newsfeed?page=1"
response = requests.get(url)
response

<Response [403]>

### [Optional] Modify headers for GET request
Having trouble with your GET request, and getting a 403 Response? 

No worries, we got you. Sometimes, making a vanilla GET request leads to this response because the server things you're not legit. 

As such, you'll need to configure your call with by supplying a header. 

Define a header, where it is a dictionary containing a 'User-Agent' key. The value? 

Hint: It's the specs of a browser.

<details>
<summary>Click once for a massive hint</summary>
<div><strong>Google "How to use Python requests to fake a browser visit a.k.a and generate User Agent?"</strong></div>
</details>

In [3]:
# Add a headers option and make the GET request
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

In [4]:
response = requests.get(url,headers=headers)
response

<Response [200]>

### Step 3: Load the response text into a BeautifulSoup object
Once you have made a successful GET request and have the response, load the response text into a BeautifulSoup object.

If you check the BeautifulSoup object, you should see something like this:

![BeautifulSoupText](https://uplevelsg.s3-ap-southeast-1.amazonaws.com/ProjectTextSummarizer/BeautifulSoupText.png)

In [5]:
# Step 3: Load the response text into a BeautifulSoup object
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.text,'html.parser')

### Step 4: Check out the HTML of the listings page
Let's go to the first page of the news. <strong>Make sure you're using Google Chrome.</strong>

When you're there, hover over the title of the first news. Right click, and click "Inspect".

If this is the first time you've tried it, you're in for a treat:

![HTMLBreakdown](https://uplevelsg.s3-ap-southeast-1.amazonaws.com/ProjectTextSummarizer/HTMLBreakdown.png)

What's happening here?

Think of your webpage as composed of boxes within boxes within boxes. 

In the Newsweek page, you have one big box, containing 30 <font color='red'>boxes</font> belonging to the "article" HTML tag. 

In each of the <font color='red'>box</font>, you have smaller boxes.
- <font color='green'>Image</font> (div)
- <font color='blue'>A box</font> (div), containing
  - <font color='yellow'>Category</font> (div)
  - <font color='purple'>Title and URL to article</font> (h3)
  - <font color='brown'>Summary</font> (div)

We'll retrieve a list containing 30 boxes first, then rummage in the first box to locate the <font color='yellow'>Category</font>, <font color='purple'>Title and URL to article</font>, and <font color='brown'>Summary</font>.

### Step 5: Find all article tags in the BeautifulSoup object
Let's use the .find_all method to get a list containing all "article" tags.

You should end up with a list containing 30 items.

In [6]:
# Step 5: Get list of "article" HTML tags
articlelist = soup.find_all('article')
#articlelist

### Step 6: Get the first item in the list
Declare a variable and assign the first item in the list to it.

We'll start dissecting the first item first before creating a loop - we do have to identify where each piece of information is after all.

The item starts with ```<article class="l3">``` and ends with ```</article>```.

In [7]:
# Step 6: Get the first item
firstarticle = articlelist[0]
firstarticle

<article class="l3"><div class="image">
<picture class="mapping-small" height="311" width="466">
<source srcset="https://d.newsweek.com/en/full/1829666/joe-biden-stimulus-check-monthly-fourth.webp?w=466&amp;h=311&amp;l=68&amp;t=41&amp;f=55e07178a7f697464af03e2eb0fddc9a" type="image/webp"/><img alt="" class="mapping-small" height="311" loading="lazy" src="https://d.newsweek.com/en/full/1829666/joe-biden-stimulus-check-monthly-fourth.jpg?w=466&amp;h=311&amp;l=68&amp;t=41&amp;f=55e07178a7f697464af03e2eb0fddc9a" width="466"/>
</picture>
<a class="zero" href="/fourth-stimulus-check-update-2000-monthly-payment-petition-hits-24-million-signatures-1603975">Fourth Stimulus Check Update: $2,000 Monthly Payment Petition Gains Support</a></div><div class="inner"><div class="category"><a href="/us">U.S.</a></div><h3><a href="/fourth-stimulus-check-update-2000-monthly-payment-petition-hits-24-million-signatures-1603975">Fourth Stimulus Check Update: $2,000 Monthly Payment Petition Gains Support</a><

### Step 7: Find "category"
First up, let's look for the "category". 

Use the .find method on your first item to look for the div tag that belongs to the "category" class. 

Once you find the div, get only the text.

In [8]:
# Step 7a: Find "category"
firstarticle.find('div',{'class':'category'})

# Step 7b: Get only the text
firstarticle.find('div',{'class':'category'}).get_text()

'U.S.'

### Step 8: Find the URL to article
The URL to the full article text contains the string "/a-news-article-that-you-attach-to-the-end". You'll have to dig to find the correct HTML tag.

<details>
<summary>Click here once if you're stuck</summary>
<div><strong>Google "get href from anchor tag beautifulsoup"</strong></div>
</details>

In [9]:
# Step 8: Find the URL
soup.find_all('h3')
#find_all_a

[<h3><a href="/fourth-stimulus-check-update-2000-monthly-payment-petition-hits-24-million-signatures-1603975">Fourth Stimulus Check Update: $2,000 Monthly Payment Petition Gains Support</a></h3>,
 <h3><a href="/dave-portnoy-shouts-fox-business-host-air-over-meme-stock-trading-1603994">Dave Portnoy Shouts At Fox Business Host On Air Over Meme Stock Trading</a></h3>,
 <h3><a href="/teacher-forced-remove-giant-swastika-pool-bottom-1603961">Teacher Forced to Remove Giant Swastika from Pool Bottom</a></h3>,
 <h3><a href="/mcconnell-biden-isnt-serious-about-bipartisanship-after-infrastructure-bill-veto-threat-1603996">McConnell Says Biden Isn't 'Serious' About Bipartisanship After Veto Threat</a></h3>,
 <h3><a href="/hawley-condemns-canadas-arrest-church-leaders-asks-us-support-1603921">Hawley Condemns Canada's Clampdown on Churches During COVID-19</a></h3>,
 <h3><a href="/20-highest-grossing-disney-movies-all-time-star-wars-frozen-1600957">20 Highest Grossing Disney Movies of All Time</a></

In [10]:
len(soup.find_all('h3'))

30

### Step 9: Find the title
The title of the article is found in the H3 tag - retrieve it! 

In [11]:
firstarticle.find('h3')

<h3><a href="/fourth-stimulus-check-update-2000-monthly-payment-petition-hits-24-million-signatures-1603975">Fourth Stimulus Check Update: $2,000 Monthly Payment Petition Gains Support</a></h3>

In [12]:
# Step 9: Get the title of the article in the first box
soup.get('a')

### Step 10: Find the summary
You'll have to find the summary text as well.

Repeat what you did for Step 7, since the summary is hidden in the div with "summary" class.

In [13]:
# Step 10a: Find the div with the "summary" class
firstarticle.find('div',{'class':'summary'})
# Step 10b: Get the text
firstarticle.find('div',{'class':'summary'}).get_text()

'If the petition hits its goal of 3 million signatures, it will become one of the most signed petitions on Change.org.'

### Step 11: Loop through all of the 30 boxes
Well done, you. You have successfully identified how to retrieve the four pieces of information that we need from one box.

Declare four empty lists, and let's build a for loop to loop through each item from the list you got from Step 5.

Append the four items to their appropriate list - we'll need them to build a DataFrame later.

<details>
<summary>Stuck? Click here once for pseudocode</summary>
<ol>
  <li>Declare four empty lists - one for category, url, title, and summary</li>
  <li>Use a for loop to loop through the list you got from Step 5. In each loop:</li>
  <ul>
    <li>Find the category of the box, and append into its list</li>
    <li>Find the URL of the main article, and append into its list</li>
    <li>Find the title of the box, and append it into its list</li>
    <li>Find the summary of the box, and append it into its list</li>
  </ul>
</ol>
</details>

In [14]:
# Step 11: Loop through all the 30 boxes and append the four items into lists
from bs4 import BeautifulSoup, SoupStrainer
category = []
URL = []
title = []
summary = []
for div in soup.find_all('div', attrs={'class':'category'}):
    category.append(div.text)
for div in soup.find_all('h3'):
    URL.append(div.find('a')['href'])
for div in soup.find_all('h3'):
    title.append(div.text)
for div in soup.find_all('div', attrs={'class':'summary'}):
    summary.append(div.text)

In [15]:
len(category),len(URL),len(title),len(summary)

(30, 30, 30, 30)

### Step 12: Build a test DataFrame
With the four lists of 30 items each, build a DataFrame. You'll see something like this:

![TestDataFrame](https://uplevelsg.s3-ap-southeast-1.amazonaws.com/ProjectTextSummarizer/TestDataFrame.png)

Expect:
- 30 rows
- 4 columns

<strong>Don't panic if your DataFrame looks differently from ours, since news get updated all the time.</strong>

In [16]:
# Step 12: Build a DataFrame with the lists
dfl = {'category':category,'url':URL,'title':title,'summary':summary}
df = pd.DataFrame(dfl)
df

,category,url,title,summary
0,U.S.,/fourth-stimulus-check-update-2000-monthly-pay...,"Fourth Stimulus Check Update: $2,000 Monthly P...",If the petition hits its goal of 3 million sig...
1,News,/dave-portnoy-shouts-fox-business-host-air-ove...,Dave Portnoy Shouts At Fox Business Host On Ai...,"""You're being a moron,"" Barstool Sports founde..."
2,World,/teacher-forced-remove-giant-swastika-pool-bot...,Teacher Forced to Remove Giant Swastika from P...,A Brazilian history teacher has been forced to...
3,News,/mcconnell-biden-isnt-serious-about-bipartisan...,McConnell Says Biden Isn't 'Serious' About Bip...,When asked if he would sign the infrastructure...
4,News,/hawley-condemns-canadas-arrest-church-leaders...,Hawley Condemns Canada's Clampdown on Churches...,Missouri Senator Josh Hawley released a letter...
5,Culture,/20-highest-grossing-disney-movies-all-time-st...,20 Highest Grossing Disney Movies of All Time,"""Star Wars,"" ""Toy Story,"" Avengers, plus both ..."
6,U.S.,/brandeis-lists-trigger-warning-among-violent-...,Brandeis Says 'Trigger Warning' Is Violent Wor...,The list of other violent and oppressive words...
7,News,/radio-host-danny-parkins-raises-660k-nonprofi...,Radio Host Raises $660k for Nonprofit Supermar...,"""You guys crushed it,"" Parkins said on air Thu..."
8,Opinion,/why-im-leaving-mumford-sons-opinion-1603978,Why I'm Leaving Mumford & Sons,The only way forward for me is to leave the ba...
9,News,/statue-george-floyd-vandalized-day-before-cha...,Statue of George Floyd Vandalized Day Before C...,"According to police, Floyd's face and the insc..."


# Scrape 50 pages of listings
Now that we've figured out how to scrape one page, let's scrape 50 pages.

At the end, we'll have 1,500 rows of data.

### Step 13: Repeat Steps 2-12 and scale to 50 pages
Modify Step 12 such that you acquire the listings information from 50 pages, instead of one page alone. 

You'll need to have two for loops instead of one, where you will exploit a URL pattern to move from one page to the next.

<details>
<summary>Stuck AGAIN? Click here once for pseudocode</summary>
<ol>
  <li>Declare four empty lists - one for category, url, title, and summary</li>
  <li>Declare a base URL for each page</li>
  <li>Use a for loop to loop a range from 1 to 51. In each loop:</li>
  <ol type ='a'>
    <li>Declare a variable containing the URL for the page to be scraped, page_num=???</li>
    <li>Make a GET request (don't forget the headers)</li>
    <li>Store the text of the GET response in a BeautifulSoup object</li>
    <li>Get a list of all the "article" tags</li>    
    <li>Use a for loop to loop through the list of articles. In each for loop:</li>
    <ul>
      <li>Find the category of the box, and append into its list</li>
      <li>Find the URL of the main article, and append into its list</li>
      <li>Find the title of the box, and append it into its list</li>
      <li>Find the summary of the box, and append it into its list</li>
    </ul>
  </ol>
</details>

In [38]:
# Step 13: Loop through 50 pages of listings and store category, URL, title, and summary in lists
category = []
URL = []
title = []
summary = []
page = range(1,51)
url = "https://www.newsweek.com/newsfeed?page=1".format(i)
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}
response = requests.get(url,headers=headers)
soup = BeautifulSoup(response.text,'html.parser')
articlelist = soup.find_all('article')
for i in page:
    url
    response
    soup
    for div in soup.find_all('div', attrs={'class':'category'}):
        category.append(div.text)
    for div in soup.find_all('h3'):
        URL.append(div.find('a')['href'])
    for div in soup.find_all('h3'):
        title.append(div.text)
    for div in soup.find_all('div', attrs={'class':'summary'}):
        summary.append(div.text)

In [39]:
len(category),len(URL),len(title),len(summary)

1500

### Step 14: Create a DataFrame with the four lists
After you scrape all of the information you need from the 50 pages, create a DataFrame like you did in Step 12.

Your DataFrame will contain the following:
- 1,500 rows
- 4 columns

In [42]:
# Step 14: Create a DataFrame with the lists
dfl = {'category':category,'url':URL,'title':title,'summary':summary}
df = pd.DataFrame(dfl)
df

,category,url,title,summary
0,U.S.,/fourth-stimulus-check-update-2000-monthly-pay...,"Fourth Stimulus Check Update: $2,000 Monthly P...",If the petition hits its goal of 3 million sig...
1,News,/dave-portnoy-shouts-fox-business-host-air-ove...,Dave Portnoy Shouts At Fox Business Host On Ai...,"""You're being a moron,"" Barstool Sports founde..."
2,World,/teacher-forced-remove-giant-swastika-pool-bot...,Teacher Forced to Remove Giant Swastika from P...,A Brazilian history teacher has been forced to...
3,News,/mcconnell-biden-isnt-serious-about-bipartisan...,McConnell Says Biden Isn't 'Serious' About Bip...,When asked if he would sign the infrastructure...
4,News,/hawley-condemns-canadas-arrest-church-leaders...,Hawley Condemns Canada's Clampdown on Churches...,Missouri Senator Josh Hawley released a letter...
...,...,...,...,...
1495,News,/ncaa-could-keep-rules-banning-pay-play-recrui...,NCAA Could Keep Rules Banning Recruiting Incen...,The NCAA is working toward a solution for name...
1496,News,/one-us-largest-unions-votes-make-amazon-prior...,One of U.S. Largest Unions Votes to Make Amazo...,"The International Brotherhood of Teamsters, wh..."
1497,Culture,/viral-therapist-shows-audacity-men-dating-app...,Viral Therapist Shows 'Audacity' of Men on Dat...,The TikTok community can't get enough of Katie...
1498,Politics,/trump-calls-mcbrooms-michigan-election-report...,Trump Calls McBroom's Michigan Election Report...,"In the statement, the former president also re..."


### Step 15: Export the DataFrame as CSV
Time to export this CSV so you can refer to it later in case you pause in the middle of this notebook.

If you're on Google Colab, make sure your Drive is properly mounted first. 

In [43]:
# Step 15: Export DataFrame as CSV
df.to_csv(r'C:\Users\Aeriu\Desktop\CourseMaterials\newsdf.csv', index = False)

# Test scrape one full article site
Time to go into one article page for scraping. 

We'll go into the first article, and inspect the HTML elements. Again, don't worry if your first article is different from ours. 

The structure of the page generally is the same throughout, and looks something like this:

![ArticleHTML](https://uplevelsg.s3-ap-southeast-1.amazonaws.com/ProjectTextSummarizer/ArticleHTML.png)

In our example, you can take a look at:
1. <font color='red'>div</font> that contains all the article content
2. <font color='green'>p</font> that contains one paragraph of text
3. <font color='blue'>div</font> that contains an ad (not shown)
4. <font color='yellow'>p</font> that contains another paragraph of text
5. <font color='red'>figure</font> that contains interactive content, e.g., a Twitter embed

This goes on for the entire article.

### Step 16: Get the URL for the first article
Before we do anything,  let's get the full URL of the first article on your DataFrame from Step 14.

The string in the url column goes something like "/xxx-xxx-xxx-xxxxxx", so you'll have to perform string concatenation with 'https://newsweek.com' to go to the article.

Assign it to a variable.

In [44]:
df

,category,url,title,summary
0,U.S.,/fourth-stimulus-check-update-2000-monthly-pay...,"Fourth Stimulus Check Update: $2,000 Monthly P...",If the petition hits its goal of 3 million sig...
1,News,/dave-portnoy-shouts-fox-business-host-air-ove...,Dave Portnoy Shouts At Fox Business Host On Ai...,"""You're being a moron,"" Barstool Sports founde..."
2,World,/teacher-forced-remove-giant-swastika-pool-bot...,Teacher Forced to Remove Giant Swastika from P...,A Brazilian history teacher has been forced to...
3,News,/mcconnell-biden-isnt-serious-about-bipartisan...,McConnell Says Biden Isn't 'Serious' About Bip...,When asked if he would sign the infrastructure...
4,News,/hawley-condemns-canadas-arrest-church-leaders...,Hawley Condemns Canada's Clampdown on Churches...,Missouri Senator Josh Hawley released a letter...
...,...,...,...,...
1495,News,/ncaa-could-keep-rules-banning-pay-play-recrui...,NCAA Could Keep Rules Banning Recruiting Incen...,The NCAA is working toward a solution for name...
1496,News,/one-us-largest-unions-votes-make-amazon-prior...,One of U.S. Largest Unions Votes to Make Amazo...,"The International Brotherhood of Teamsters, wh..."
1497,Culture,/viral-therapist-shows-audacity-men-dating-app...,Viral Therapist Shows 'Audacity' of Men on Dat...,The TikTok community can't get enough of Katie...
1498,Politics,/trump-calls-mcbrooms-michigan-election-report...,Trump Calls McBroom's Michigan Election Report...,"In the statement, the former president also re..."


In [57]:
# Step 16: Get the URL for the first article
nw = 'https://newsweek.com'
firsturl = nw + df['url'][0]
firsturl

'https://newsweek.com/fourth-stimulus-check-update-2000-monthly-payment-petition-hits-24-million-signatures-1603975'

### Step 17: Repeat Steps 2-4 on the first article
Let's repeat Steps 2-4 on the article URL. To recap, you'll need to:
1. Make a GET request
2. Load the GET response into a BeatifulSoup object
3. Check out the HTML of the article page

In [162]:
# Step 17a: Perform a GET request on the article URL
response = requests.get(firsturl,headers=headers)
response
# Step 17b: Load the response into a BeautifulSoup object, in a new variable
firstsoup = BeautifulSoup(response.text,'html.parser')
# Step 17c: Check out the HTML in the BeautifulSoup
#firstsoup.prettify()

### Step 18: Find the div containing all article content
We'll look for the <font color='red'>div</font> that contains all of our article text (scroll up a bit for reference).

More specifically, look for the div that belongs to a class named 'article-body v_text paywall'.

In [67]:
# Step 18: Get the div
firstsoup.find('div',{'class':'article-body v_text paywall'}).prettify()

'<div class="article-body v_text paywall">\n <p>\n  A petition for monthly stimulus checks surpassed 2.4 million signatures on Thursday, putting it only about 600,000 signatures away from its target goal of 3 million.\n </p>\n <p>\n  Stephanie Bonin, a Denver restaurant owner, started the petition last year when the pandemic forced the closure of businesses across the country. It\'s had a steady stream of supporters and despite states reopening businesses and lifting restrictions, people continue to sign the petition that urges\n  <a data-sys="1" href="https://www.newsweek.com/topic/congress">\n   Congress\n  </a>\n  to take additional action.\n </p>\n <p>\n  The\n  <a href="https://www.change.org/p/give-2000-month-to-every-american-moneyforthepeople-covid19" rel="nofollow">\n   petition\n  </a>\n  calls for Congress to pass relief that would send out $2,000 monthly payments to adults and $1,000 payments for kids. It urges legislators to continue the payments for the "duration of the c

### Step 19: Get all p only
As you scroll through the article HTML, what do you notice? The text can found in only <font color='green'>p</font> HTML tags.

Use the .find_all method to find only <font color='green'>p</font>.

You'll end up with a list, something like this:
![ListOfParagraphs](https://uplevelsg.s3-ap-southeast-1.amazonaws.com/ProjectTextSummarizer/ListOfParagraphs.png)

In [71]:
# Step 19: Find all p tags in your BeautifulSoup
firstsoup.find_all('p')

[<p>A petition for monthly stimulus checks surpassed 2.4 million signatures on Thursday, putting it only about 600,000 signatures away from its target goal of 3 million.</p>,
 <p>Stephanie Bonin, a Denver restaurant owner, started the petition last year when the pandemic forced the closure of businesses across the country. It's had a steady stream of supporters and despite states reopening businesses and lifting restrictions, people continue to sign the petition that urges <a data-sys="1" href="https://www.newsweek.com/topic/congress">Congress</a> to take additional action.</p>,
 <p>The <a href="https://www.change.org/p/give-2000-month-to-every-american-moneyforthepeople-covid19" rel="nofollow">petition</a> calls for Congress to pass relief that would send out $2,000 monthly payments to adults and $1,000 payments for kids. It urges legislators to continue the payments for the "duration of the crisis" to help people who were financially impacted by the pandemic pay their rent and put fo

### Step 20: Join all of the text together
You have a list of p tags containing a mix of text and URLs.

Extract all of the text from the p tags with the .text attribute and join them together into a long string.

<details>
<summary>Click here once for a small hint</summary>
<div><strong>You'll need a for loop so you can append only the text from the p tags into a list, and a .join method at the end to join all of the strings in the list into a larger string.</strong></div>
</details>

In [201]:
type(firsturl),type(urllist)

(str, pandas.core.series.Series)

In [196]:
#response = requests.get(urllist,headers=headers)

In [218]:
# Step 20: Join all of the text together
pd.set_option('display.max_colwidth', None)
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}
nw = 'https://newsweek.com'
urllist = nw + df['url']
for links in urllist:
    response = requests.get(links,headers=headers)
newsoup = BeautifulSoup(response.text,'html.parser')
ptext = []
ptextlong = []
for div in newsoup.find_all('p'):
    ptext.append(div.text)
ptextlong.append(' '.join(ptext))

In [219]:
len(ptext),len(ptextlong)

(26, 1)

In [220]:
ptextlong

['Weed cookies resembling Oreos and Chips Ahoy brand cookies sold in Long Beach, New York, are prompting concerns over "targeting children," New York Senator Todd Kaminsky said. These treats containing synthetic marijuana are marketed as "Double Stuf Stoneos" and "Trips Ahoy" with packaging extremely similar to the famous snack brands they appear to be based on, according to WABC. They were found at a wellness store named Dr. Nature in Long Beach. @pconnection1  So how far is too far on Hemp-Extract products? Oreo vs Stoneo, and Chips Ahoy vs Trips Ahoy.#HempProduct#WeedLovers pic.twitter.com/bUWeglp4Dv "This is extremely discouraging and alarming to know that these products could end up in the hands of our young children who think they\'re having an afternoon snack. That cannot be allowed," he also said. The cookies contain Delta-8, a psychoactive, hemp-based substance found in cannabis plants that is legal in New York. Dr. Nature\'s shop owner reportedly did not know the packaged coo

In [217]:
len(ptext),len(ptextlong)

(0, 1)

In [222]:
ptext

['Weed cookies resembling Oreos and Chips Ahoy brand cookies sold in Long Beach, New York, are prompting concerns over "targeting children," New York Senator Todd Kaminsky said.',
 'These treats containing synthetic marijuana are marketed as "Double Stuf Stoneos" and "Trips Ahoy" with packaging extremely similar to the famous snack brands they appear to be based on, according to WABC. They were found at a wellness store named Dr. Nature in Long Beach.',
 '@pconnection1  So how far is too far on Hemp-Extract products? Oreo vs Stoneo, and Chips Ahoy vs Trips Ahoy.#HempProduct#WeedLovers pic.twitter.com/bUWeglp4Dv',
 '"This is extremely discouraging and alarming to know that these products could end up in the hands of our young children who think they\'re having an afternoon snack. That cannot be allowed," he also said.',
 'The cookies contain Delta-8, a psychoactive, hemp-based substance found in cannabis plants that is legal in New York.',
 "Dr. Nature's shop owner reportedly did not kn

In [158]:
ptextlong

['A petition for monthly stimulus checks surpassed 2.4 million signatures on Thursday, putting it only about 600,000 signatures away from its target goal of 3 million. Stephanie Bonin, a Denver restaurant owner, started the petition last year when the pandemic forced the closure of businesses across the country. It\'s had a steady stream of supporters and despite states reopening businesses and lifting restrictions, people continue to sign the petition that urges Congress to take additional action. The petition calls for Congress to pass relief that would send out $2,000 monthly payments to adults and $1,000 payments for kids. It urges legislators to continue the payments for the "duration of the crisis" to help people who were financially impacted by the pandemic pay their rent and put food on the table. If the petition reaches its goal of 3 million signatures, it will be one of the most-signed petitions on the Change.org website. In December, Bonin was recognized for having started o

# Scrape all 1,500 articles
Now that you know how to scrape the text from one article, it's time to loop through the url column of your DataFrame from Step 14.

### Step 21: Scrape all articles from url column
Create a list containing the text from all 1,500 articles. 

It's repeating Steps 16-20, but appending the results into a list. 

Use a try-except conditiional to catch any unexpected errors, and return None. Our list MUST have 1,500 items in it.

<strong>This step will take between 6 to 10 minutes - be patient.</strong>

In [164]:
# Step 21: Scrape the articles found in the url column
ptext = []
ptextlong = []
nw = 'https://newsweek.com'
for i in df:
    ptext.append(nw + df['url'])

In [165]:
ptext

[0       https://newsweek.com/fourth-stimulus-check-upd...
 1       https://newsweek.com/dave-portnoy-shouts-fox-b...
 2       https://newsweek.com/teacher-forced-remove-gia...
 3       https://newsweek.com/mcconnell-biden-isnt-seri...
 4       https://newsweek.com/hawley-condemns-canadas-a...
                               ...                        
 1495    https://newsweek.com/ncaa-could-keep-rules-ban...
 1496    https://newsweek.com/one-us-largest-unions-vot...
 1497    https://newsweek.com/viral-therapist-shows-aud...
 1498    https://newsweek.com/trump-calls-mcbrooms-mich...
 1499    https://newsweek.com/weed-cookies-that-resembl...
 Name: url, Length: 1500, dtype: object,
 0       https://newsweek.com/fourth-stimulus-check-upd...
 1       https://newsweek.com/dave-portnoy-shouts-fox-b...
 2       https://newsweek.com/teacher-forced-remove-gia...
 3       https://newsweek.com/mcconnell-biden-isnt-seri...
 4       https://newsweek.com/hawley-condemns-canadas-a...
               

### Step 22: Create full_text column
With the list you got from Step 21, create a new column named full_text.

You'll end up with something like this:

![FullTextColumn](https://uplevelsg.s3-ap-southeast-1.amazonaws.com/ProjectTextSummarizer/FullTextColumn.png)
You'll have:
1. 1,500 rows
2. 5 columns

In [ ]:
# Step 22: Create full_text column

### Step 23: Export the DataFrame
With the DataFrame complete with both the summary and full text of the article, let's export the DataFrame for subsequent Parts.

In [ ]:
# Step 23: Export the DataFrame as CSV

# End of Part I
Phew, what a Part. Hope you enjoyed the data collection segment. The thing about web scraping is that once you get the hang of it, you'll feel like you're wielding a new superpower.

In this Part, you scraped both the listings page of Newsweek and the article text within. 

In the next Part, we'll start with extractive summarization and summarize the 1,500 articles that you've scraped.

<strong>Note: we'll be using our own dataset from here onwards - if you'd like to use the same dataset as us for consistent reference, just click <a href="https://uplevelsg.s3-ap-southeast-1.amazonaws.com/df_part_i.csv">here</a> to download the CSV.</strong>